# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

EL OBJETIVO DE LA LIBRERIA SQLALCHEMY ES HACER DE TRADUCTOR DE SINTAXIS ENTRE SERVIDORES SQL,SUS COMANDOS Y CODIGO PYHTON.
PRIMERO DE TODO INSTALAMOS LAS DEPENDENCIAS MEDIANTE TERMINAL GIT CON PIP.
pip install sqlalchemy python-dotenv psycopg2-binary.
SI hay errores de incompatibilidad tendremos que utilizar diferentes versiones de python hasta encontrar una que no haya errores de compatibilidad entre las dependencias de cada libreria.
UTILIZARE MI PROPIO SERVIDOR POSTGRESQL INSTALADO EN MI SOBREMESA Y PREVIAMENTE CONECTADO MEDIANTE SSL, PREVIAMENTE CONFIGURADO CON PGADMIN4.
LAS CREDENCIALES SE ENLAZARAN A LA LLAMADA DE UNA VARIABLE EXTERNA A ESTE DOCUMENTO, EL MISMO SERA INVISIBLE PARA LOS USUARIOS DE ESTE WORKSPACE EN REMOTO.
PARA ELLO SE ANOTARA EL NOMBRE DEL DOCUMENTO EN EL .GITIGNORE, EL ARCHIVO EN QUESTION ES UN .ENV QUE SE UTILIZA BASICAMENTE COMO ARCHIVO DE CONFIGURACION.
PARA OPERAR CON ESTE TIPO DE ARCHIVO SE UTILIZA LA LIBRERIA DOTENV COMUNMENTE.

In [6]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String, Date, DECIMAL, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from pathlib import Path
current_dir = Path(os.getcwd())
dotenv_path = current_dir / 'user.env'
load_dotenv(dotenv_path=dotenv_path)
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)
Base = declarative_base()

Session = sessionmaker(bind=engine)
session = Session()
class Publisher(Base):
    __tablename__ = 'publishers'
    publisher_id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)

class Author(Base):
    __tablename__ = 'authors'
    author_id = Column(Integer, primary_key=True)
    first_name = Column(String(100), nullable=False)
    middle_name = Column(String(50))
    last_name = Column(String(100))

class Book(Base):
    __tablename__ = 'books'
    book_id = Column(Integer, primary_key=True)
    title = Column(String(255), nullable=False)
    total_pages = Column(Integer)
    rating = Column(DECIMAL(4, 2))
    isbn = Column(String(13))
    published_date = Column(Date)
    publisher_id = Column(Integer, ForeignKey('publishers.publisher_id'))
    publisher = relationship("Publisher")

class BookAuthor(Base):
    __tablename__ = 'book_authors'
    book_id = Column(Integer, ForeignKey('books.book_id'), primary_key=True)
    author_id = Column(Integer, ForeignKey('authors.author_id'), primary_key=True)
    book = relationship("Book")
    author = relationship("Author")
Base.metadata.create_all(engine)
if not session.query(Publisher).first():
    publishers_data = [
        
    ]
    session.add_all(publishers_data)

if not session.query(Author).first():
    authors_data = [
       
    ]
    session.add_all(authors_data)

if not session.query(Book).first():
    books_data = [
        
    ]
    session.add_all(books_data)

if not session.query(BookAuthor).first():
    book_authors_data = [
        
    ]
    session.add_all(book_authors_data)

session.commit()

In [7]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from pathlib import Path
current_dir = Path(os.getcwd())
dotenv_path = current_dir / 'user.env'
load_dotenv(dotenv_path=dotenv_path)
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)

Session = sessionmaker(bind=engine)
session = Session()
session.execute("""
    CREATE TABLE IF NOT EXISTS unified_books AS
    SELECT
        b.book_id,
        b.title,
        b.total_pages,
        b.rating,
        b.isbn,
        b.published_date,
        p.name AS publisher_name,
        a.first_name,
        a.middle_name,
        a.last_name
    FROM
        books b
    LEFT JOIN
        publishers p ON b.publisher_id = p.publisher_id
    LEFT JOIN
        book_authors ba ON b.book_id = ba.book_id
    LEFT JOIN
        authors a ON ba.author_id = a.author_id;
""")
session.commit()

In [8]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from pathlib import Path

# Obtener la ruta absoluta del directorio actual (para Jupyter)
current_dir = Path(os.getcwd())

# Construir la ruta absoluta al archivo user.env
dotenv_path = current_dir / 'user.env'

# Cargar variables de entorno desde la ruta absoluta
load_dotenv(dotenv_path=dotenv_path)

# Configuración de la conexión a PostgreSQL
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)

# Paso 1: Ejecutar las consultas SQL y obtener los resultados
with engine.connect() as connection:
    publishers_results = connection.execute("SELECT * FROM publishers").fetchall()
    authors_results = connection.execute("SELECT * FROM authors").fetchall()
    books_results = connection.execute("SELECT * FROM books").fetchall()
    book_authors_results = connection.execute("SELECT * FROM book_authors").fetchall()
    unified_books_results = connection.execute("SELECT * FROM unified_books").fetchall()

# Paso 2: Convertir los resultados a DataFrames de pandas
df_publishers = pd.DataFrame(publishers_results, columns=["publisher_id", "name"])
df_authors = pd.DataFrame(authors_results, columns=["author_id", "first_name", "middle_name", "last_name"])
df_books = pd.DataFrame(books_results, columns=["book_id", "title", "total_pages", "rating", "isbn", "published_date", "publisher_id"])
df_book_authors = pd.DataFrame(book_authors_results, columns=["book_id", "author_id"])
df_unified_books = pd.DataFrame(unified_books_results, columns=["book_id", "title", "total_pages", "rating", "isbn", "published_date", "publisher_name", "first_name", "middle_name", "last_name"])

# Imprimir los DataFrames
print(df_publishers.head(2), '\n')
print("-" * 50)
print(df_authors.head(2), '\n')
print("-" * 50)
print(df_books.head(2), '\n')
print("-" * 50)
print(df_book_authors.head(2), '\n')
print("-" * 50)
print(df_unified_books.head(2), '\n')
print("-" * 50)

Empty DataFrame
Columns: [publisher_id, name]
Index: [] 

--------------------------------------------------
Empty DataFrame
Columns: [author_id, first_name, middle_name, last_name]
Index: [] 

--------------------------------------------------
Empty DataFrame
Columns: [book_id, title, total_pages, rating, isbn, published_date, publisher_id]
Index: [] 

--------------------------------------------------
Empty DataFrame
Columns: [book_id, author_id]
Index: [] 

--------------------------------------------------
Empty DataFrame
Columns: [book_id, title, total_pages, rating, isbn, published_date, publisher_name, first_name, middle_name, last_name]
Index: [] 

--------------------------------------------------
